In [0]:
36## Ha Choi 2016 Paper
## Doi : 10.1109/IJCNN.2016.7727224

In [0]:
##Libraries
import sys
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st

from keras.layers import Input, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, Activation, concatenate, merge
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback
from keras.models import Model
from keras import backend as K
K.set_image_data_format('channels_first')
import warnings
warnings.filterwarnings('ignore')

In [0]:
!pip install tensorflow==1.5

In [0]:
!pip install numpy==1.16.1
!pip install keras==2.1.2

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("/content/drive/")
!ls

'My Drive'


In [0]:
import os
os.chdir("My Drive/Computer_Vision_Masters/Wearable_Sensors_Code/")

In [0]:
def custom_model(X, idx_modalities, n_classes):
    img_cols1 = idx_modalities[0]
    img_cols2 = idx_modalities[1] - idx_modalities[0]
    img_cols3 = idx_modalities[2] - idx_modalities[1]
    img_cols4 = X.shape[3] - idx_modalities[2]

    _, _, img_rows, img_cols = X.shape
    inp_modality1 = Input((1, img_rows, img_cols1))
    inp_modality2 = Input((1, img_rows, img_cols2))
    inp_modality3 = Input((1, img_rows, img_cols3))
    inp_modality4 = Input((1, img_rows, img_cols4))

    H1 = Conv2D(filters=5, kernel_size=(5, 5))(inp_modality1)
    H1 = Activation('relu')(H1)
    H1 = MaxPooling2D(pool_size=(4, 4))(H1)

    H2 = Conv2D(filters=5, kernel_size=(5, 5))(inp_modality2)
    H2 = Activation('relu')(H2)
    H2 = MaxPooling2D(pool_size=(4, 4))(H2)

    H3 = Conv2D(filters=5, kernel_size=(5, 5))(inp_modality3)
    H3 = Activation('relu')(H3)
    H3 = MaxPooling2D(pool_size=(4, 4))(H3)

    # H4 = Conv2D(filters=5, kernel_size=(5, 3))(inp_modality4)#For PAMAP
    H4 = Conv2D(filters=5, kernel_size=(5, 2))(inp_modality4)  # For MEHEALTH
    H4 = Activation('relu')(H4)
    H4 = MaxPooling2D(pool_size=(4, 1))(H4)

    shape_1 = int(H2.shape[1].value)
    shape_2 = int(H2.shape[2].value)
    shape_3 = int(H2.shape[3].value)
    inp_zeros = Input((shape_1, shape_2, shape_3))  # Here is the features map shape

    H = concatenate([H1, inp_zeros, H2, inp_zeros, H3, inp_zeros, H4], axis=3)

    H = Conv2D(filters=10, kernel_size=(5, 5))(H)
    H = Activation('relu')(H)
    H = MaxPooling2D(pool_size=(2, 2))(H)

    H = Flatten()(H)
    H = Dense(120)(H)
    H = Activation('relu')(H)

    H = Dense(n_classes)(H)
    H = Activation('softmax')(H)

    model = Model([inp_modality1, inp_modality2, inp_modality3, inp_modality4, inp_zeros], H)

    return model, (shape_1, shape_2, shape_3)

In [0]:
def zero_padding_MHEALTH(X):
    # Groups the heterogenous sensors for MHEALTH
    idx_modalities = []
    idx_acc = [0, 1, 2, 5, 6, 7, 14, 15, 16]
    idx_gyro = [8, 9, 10, 17, 18, 19]
    idx_mag = [11, 12, 13, 20, 21, 22]
    idx_ele = [3, 4]
    X_acc = X[:, :, :, idx_acc]
    X_gyro = X[:, :, :, idx_gyro]
    X_mag = X[:, :, :, idx_mag]
    X_ele = X[:, :, :, idx_ele]
    X_zeros = np.zeros((X.shape[0], X.shape[1], X.shape[2], 2))  # Vertical Kernel-1 = 2

    X = X_acc
    X = np.concatenate((X, X_zeros), axis=3)
    idx_modalities.append(X.shape[3])

    X = np.concatenate((X, X_gyro), axis=3)
    X = np.concatenate((X, X_zeros),axis=3)
    idx_modalities.append(X.shape[3])

    X = np.concatenate((X, X_mag),axis=3)
    X = np.concatenate((X, X_zeros),axis=3)
    idx_modalities.append(X.shape[3])
    X = np.concatenate((X, X_ele),axis=3)

    return X, idx_modalities


In [0]:
def split_X(X, idx_modalities, zeros):
    X_tmp = []
    X_tmp.append(X[:, :, :, 0:idx_modalities[0]])
    X_tmp.append(X[:, :, :, idx_modalities[0]:idx_modalities[1]])
    X_tmp.append(X[:, :, :, idx_modalities[1]:idx_modalities[2]])
    X_tmp.append(X[:, :, :, idx_modalities[2]:X.shape[3]])
    X_tmp.append(zeros)
    return X_tmp

In [0]:
def custom_stopping(value=0.5, verbose=0):
    early = EarlyStoppingByLossVal(monitor='val_loss', value=value, verbose=verbose)
    return early

class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_acc', value=0.95, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        # if current is None:
        # warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [0]:
def DataPreparation(data_input_file):
  np.random.seed(12227)

  tmp = np.load(data_input_file)
  X = tmp['X']
  y = tmp['y']
  folds = tmp['folds']

  n_class = y.shape[1]

  X, idx_modalities = zero_padding_MHEALTH(X)

  _, _, img_rows, img_cols = X.shape
  return X, y, folds, idx_modalities,n_class

In [0]:
def TrainAndReportAccuracies(X,y,folds,idx_modalities,n_class):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  for i in range(0, len(folds)):
      train_idx = folds[i][0]
      test_idx = folds[i][1]

      X_train = X[train_idx]
      X_test = X[test_idx]

      model, inp_zeros = custom_model(X, idx_modalities, n_classes=n_class)

      zeros_mat = np.zeros((X_train.shape[0], inp_zeros[0], inp_zeros[1], inp_zeros[2]))
      model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adadelta')

      X_train = split_X(X_train, idx_modalities, zeros_mat)
      print("Start Training")
      model.fit(X_train, y[train_idx], batch_size=100, epochs=60,
                verbose=1, callbacks=[custom_stopping(value=0.2, verbose=1)], validation_data=(X_train, y[train_idx]))

      X_test = split_X(X_test, idx_modalities, zeros_mat)

      y_pred = model.predict(X_test)
      y_pred = np.argmax(y_pred, axis=1)

      y_true = np.argmax(y[test_idx], axis=1)

      acc_fold = accuracy_score(y_true, y_pred)
      avg_acc.append(acc_fold)

      recall_fold = recall_score(y_true, y_pred, average='macro')
      avg_recall.append(recall_fold)

      f1_fold = f1_score(y_true, y_pred, average='macro')
      avg_f1.append(f1_fold)

      print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold, i))
      print('______________________________________________________')
      del model
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [0]:
def Run(data_input_file):
  X, y, folds, idx_modalities,n_class= DataPreparation(data_input_file)
  TrainAndReportAccuracies(X,y,folds,idx_modalities,n_class)

In [0]:
def WisdomDataPreparation(data_input_file):
    np.random.seed(12227)

    tmp = np.load(data_input_file)
    X = tmp['X']
    y = tmp['y']
    folds = tmp['folds']

    n_class = y.shape[1]
    print(n_class)
    print(X.shape)
    print(y.shape)
    X=np.pad(X, ((0,0),(0,0), (150,0), (26, 0)), 'constant')
    print(X.shape)
    train_idx = folds[0][0]
    test_idx = folds[0][1]

    X_train = X[train_idx]
    X_test = X[test_idx]
    print(X_train.shape)
    print(X_test.shape)
    idx_modalities=[11,19,27]
    print(idx_modalities)
    return X,y,folds,idx_modalities,n_class

In [0]:
def RunWisdom(data_input_file):
  X,y,folds,idx_modalities,n_class=WisdomDataPreparation(data_input_file)
  TrainAndReportAccuracies(X,y,folds,idx_modalities,n_class)

In [0]:
def WHARFDataPreparation(data_input_file):
    np.random.seed(12227)
    tmp = np.load(data_input_file)
    X = tmp['X']
    y = tmp['y']
    folds = tmp['folds']
    X=np.pad(X, ((0,0),(0,0),(90,0), (26, 0)), 'constant')
    n_class = y.shape[1]
    print(n_class)
    print(X.shape)
    print(y.shape)

    train_idx = folds[0][0]
    test_idx = folds[0][1]

    X_train = X[train_idx]
    X_test = X[test_idx]
    print(X_train.shape)
    print(X_test.shape)

    idx_modalities=[11,19,27]
    print(idx_modalities)
    return X,y,folds,idx_modalities,n_class

In [0]:
def RunWHARF(data_input_file):
  # Batch size 100
  X,y,folds,idx_modalities,n_class=WHARFDataPreparation(data_input_file)
  TrainAndReportAccuracies(X,y,folds,idx_modalities,n_class)

In [0]:
RunWHARF('data/LOSO/WHARF.npz')

12
(3871, 1, 250, 29)
(3871, 12)
(2189, 1, 250, 29)
(1682, 1, 250, 29)
[11, 19, 27]
Start Training
Train on 2189 samples, validate on 2189 samples
Epoch 1/60
2189/2189 [==============================] - 19s 9ms/step - loss: 2.3981 - acc: 0.2015 - val_loss: 2.0687 - val_acc: 0.3851
Epoch 2/60
2189/2189 [==============================] - 19s 8ms/step - loss: 1.9891 - acc: 0.4061 - val_loss: 2.0692 - val_acc: 0.3344
Epoch 3/60
2189/2189 [==============================] - 18s 8ms/step - loss: 1.8065 - acc: 0.4463 - val_loss: 1.6765 - val_acc: 0.4632
Epoch 4/60
2189/2189 [==============================] - 18s 8ms/step - loss: 1.6324 - acc: 0.4756 - val_loss: 1.5647 - val_acc: 0.4623
Epoch 5/60
2189/2189 [==============================] - 19s 9ms/step - loss: 1.4966 - acc: 0.4902 - val_loss: 1.4403 - val_acc: 0.5190
Epoch 6/60
2189/2189 [==============================] - 19s 8ms/step - loss: 1.4134 - acc: 0.5080 - val_loss: 1.4021 - val_acc: 0.4902
Epoch 7/60
2189/2189 [=====================

In [19]:
RunWHARF('data/LOTO/WHARF.npz')

12
(3871, 1, 250, 29)
(3871, 12)
(3472, 1, 250, 29)
(399, 1, 250, 29)
[11, 19, 27]
Start Training
Train on 3472 samples, validate on 3472 samples
Epoch 1/60
3472/3472 [==============================] - 35s 10ms/step - loss: 2.2045 - acc: 0.2952 - val_loss: 1.9852 - val_acc: 0.3978
Epoch 2/60
3472/3472 [==============================] - 37s 11ms/step - loss: 1.8985 - acc: 0.4035 - val_loss: 1.7021 - val_acc: 0.4490
Epoch 3/60
3472/3472 [==============================] - 34s 10ms/step - loss: 1.6498 - acc: 0.4519 - val_loss: 1.9356 - val_acc: 0.3534
Epoch 4/60
3472/3472 [==============================] - 34s 10ms/step - loss: 1.5155 - acc: 0.4821 - val_loss: 1.4276 - val_acc: 0.5009
Epoch 5/60
3472/3472 [==============================] - 34s 10ms/step - loss: 1.3928 - acc: 0.5072 - val_loss: 1.3415 - val_acc: 0.5058
Epoch 6/60
3472/3472 [==============================] - 34s 10ms/step - loss: 1.3012 - acc: 0.5432 - val_loss: 1.2932 - val_acc: 0.5225
Epoch 7/60
3472/3472 [================

In [0]:
RunWHARF('data/FNOW/WHARF.npz')

12
(2146, 1, 250, 29)
(2146, 12)
(1926, 1, 250, 29)
(220, 1, 250, 29)
[11, 19, 27]
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Start Training
Train on 1926 samples, validate on 1926 samples
Epoch 1/60
1926/1926 [==============================] - 16s 8ms/step - loss: 2.4667 - acc: 0.1490 - val_loss: 2.2209 - val_acc: 0.2799
Epoch 2/60
1926/1926 [==============================] - 16s 8ms/step - loss: 2.1380 - acc: 0.3489 - val_loss: 2.0353 - val_acc: 0.3884
Epoch 3/60
1926/1926 [==============================] - 16s 8ms/step - loss: 1.9735 - acc: 0.3645 - val_loss: 1.8350 - val_acc: 0.4117
Epoch 4/60
1926/1926 [==============================] - 16s 8ms/step - loss: 1.8422 - acc: 0.3951 - val_loss: 2.0836 - val_acc: 0.3193
Epoch 5/60
1926/1926 [==============================] - 16s 8ms/step - loss: 1.7300 - acc: 0.42

In [18]:
RunWHARF('data/SNOW/WHARF.npz')

12
(3880, 1, 250, 29)
(3880, 12)
(3485, 1, 250, 29)
(395, 1, 250, 29)
[11, 19, 27]
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Start Training
Train on 3485 samples, validate on 3485 samples
Epoch 1/60
3485/3485 [==============================] - 35s 10ms/step - loss: 2.3026 - acc: 0.2617 - val_loss: 2.0333 - val_acc: 0.3948
Epoch 2/60
3485/3485 [==============================] - 34s 10ms/step - loss: 1.9298 - acc: 0.3963 - val_loss: 1.8302 - val_acc: 0.4080
Epoch 3/60
3485/3485 [==============================] - 34s 10ms/step - loss: 1.6876 - acc: 0.4456 - val_loss: 1.7391 - val_acc: 0.4172
Epoch 4/60
3485/3485 [==============================] - 34s 10ms/step - loss: 1.5078 - acc: 0.4783 - val_loss: 1.4072 - val_acc: 0.5128
Epoch 5/60
3485/3485 [==============================] - 34s 10ms/step - loss: 1.3747 - acc:

In [0]:
RunWisdom('data/LOSO/WISDM.npz')

6
(20846, 1, 100, 3)
(20846, 6)
(20846, 1, 250, 29)
(20167, 1, 250, 29)
(679, 1, 250, 29)
[11, 19, 27]
Start Training
Train on 20167 samples, validate on 20167 samples
Epoch 1/100
20167/20167 [==============================] - 175s 9ms/step - loss: 1.5223 - acc: 0.3451 - val_loss: 1.2309 - val_acc: 0.4439
Epoch 2/100
20167/20167 [==============================] - 174s 9ms/step - loss: 1.1885 - acc: 0.4568 - val_loss: 1.2280 - val_acc: 0.3475
Epoch 3/100
20167/20167 [==============================] - 170s 8ms/step - loss: 1.1205 - acc: 0.5000 - val_loss: 1.1111 - val_acc: 0.5158
Epoch 4/100
20167/20167 [==============================] - 169s 8ms/step - loss: 1.0325 - acc: 0.5568 - val_loss: 0.9566 - val_acc: 0.6247
Epoch 5/100
20167/20167 [==============================] - 170s 8ms/step - loss: 0.9438 - acc: 0.6316 - val_loss: 0.8675 - val_acc: 0.6743
Epoch 6/100
20167/20167 [==============================] - 171s 8ms/step - loss: 0.8581 - acc: 0.6606 - val_loss: 0.7915 - val_acc: 0.690

In [0]:
RunWisdom('data/LOTO/WISDM.npz')

6
(20846, 1, 100, 3)
(20846, 6)
(20846, 1, 250, 29)
(18393, 1, 250, 29)
(2453, 1, 250, 29)
[11, 19, 27]
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Start Training
Train on 18393 samples, validate on 18393 samples
Epoch 1/60
18393/18393 [==============================] - 150s 8ms/step - loss: 1.5524 - acc: 0.3272 - val_loss: 1.2477 - val_acc: 0.4261
Epoch 2/60
18393/18393 [==============================] - 149s 8ms/step - loss: 1.2010 - acc: 0.4466 - val_loss: 1.1513 - val_acc: 0.4627
Epoch 3/60
18393/18393 [==============================] - 148s 8ms/step - loss: 1.1527 - acc: 0.4715 - val_loss: 1.1037 - val_acc: 0.5284
Epoch 4/60
18393/18393 [==============================] - 148s 8ms/step - loss: 1.0850 - acc: 0.5110 - val_loss: 0.9811 - val_acc: 0.6024
Epoch 5/60
18393/18393 [==============================] - 14

In [0]:
RunWisdom('data/SNOW/WISDM.npz')

6
(20846, 1, 100, 3)
(20846, 6)
(20846, 1, 250, 29)
(18757, 1, 250, 29)
(2089, 1, 250, 29)
[11, 19, 27]
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Start Training
Train on 18757 samples, validate on 18757 samples
Epoch 1/60
18757/18757 [==============================] - 171s 9ms/step - loss: 1.5490 - acc: 0.3224 - val_loss: 1.2569 - val_acc: 0.4085
Epoch 2/60
18757/18757 [==============================] - 171s 9ms/step - loss: 1.2035 - acc: 0.4561 - val_loss: 1.1545 - val_acc: 0.5144
Epoch 3/60
18757/18757 [==============================] - 170s 9ms/step - loss: 1.1407 - acc: 0.4815 - val_loss: 1.1210 - val_acc: 0.4497
Epoch 4/60
18757/18757 [==============================] - 170s 9ms/step - loss: 1.0752 - acc: 0.5168 - val_loss: 1.0262 - val_acc: 0.5719
Epoch 5/60
18757/18757 [==============================] - 17

In [0]:
RunWisdom('data/FNOW/WISDM.npz')

6
(10516, 1, 100, 3)
(10516, 6)
(10516, 1, 250, 29)
(9461, 1, 250, 29)
(1055, 1, 250, 29)
[11, 19, 27]
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Start Training
Train on 9461 samples, validate on 9461 samples
Epoch 1/60
9461/9461 [==============================] - 93s 10ms/step - loss: 1.7582 - acc: 0.2605 - val_loss: 1.4194 - val_acc: 0.3845
Epoch 2/60
9461/9461 [==============================] - 92s 10ms/step - loss: 1.3210 - acc: 0.3953 - val_loss: 1.2502 - val_acc: 0.4251
Epoch 3/60
9461/9461 [==============================] - 92s 10ms/step - loss: 1.2211 - acc: 0.4398 - val_loss: 1.1890 - val_acc: 0.4521
Epoch 4/60
9461/9461 [==============================] - 92s 10ms/step - loss: 1.1661 - acc: 0.4857 - val_loss: 1.1204 - val_acc: 0.5266
Epoch 5/60
9461/9461 [==============================] - 92s 10ms/step -

In [0]:
Run('data/LOSO/MHEALTH.npz')

Start Training
Epoch 00073: early stopping THR
Accuracy[0.8706] Recall[0.8763] F1[0.8684] at fold[0]
______________________________________________________
Start Training
Epoch 00054: early stopping THR
Accuracy[0.7567] Recall[0.7495] F1[0.7311] at fold[1]
______________________________________________________
Start Training
Epoch 00081: early stopping THR
Accuracy[0.7809] Recall[0.7652] F1[0.7539] at fold[2]
______________________________________________________
Start Training
Accuracy[0.7707] Recall[0.7587] F1[0.7177] at fold[3]
______________________________________________________
Start Training
Accuracy[0.7849] Recall[0.7369] F1[0.6947] at fold[4]
______________________________________________________
Start Training
Epoch 00054: early stopping THR
Accuracy[0.7273] Recall[0.6814] F1[0.6184] at fold[5]
______________________________________________________
Start Training
Epoch 00066: early stopping THR
Accuracy[0.7589] Recall[0.7285] F1[0.7105] at fold[6]
___________________________

In [0]:
Run('data/LOTO/MHEALTH.npz')

Start Training
Epoch 00070: early stopping THR
Accuracy[0.6953] Recall[0.6797] F1[0.6700] at fold[0]
______________________________________________________
Start Training
Accuracy[0.7977] Recall[0.7463] F1[0.7131] at fold[1]
______________________________________________________
Start Training
Accuracy[0.7463] Recall[0.7355] F1[0.6710] at fold[2]
______________________________________________________
Start Training
Epoch 00060: early stopping THR
Accuracy[0.7137] Recall[0.7123] F1[0.6922] at fold[3]
______________________________________________________
Start Training
Accuracy[0.8745] Recall[0.8283] F1[0.8164] at fold[4]
______________________________________________________
Start Training
Accuracy[0.8945] Recall[0.8713] F1[0.8459] at fold[5]
______________________________________________________
Start Training
Accuracy[0.8000] Recall[0.7526] F1[0.7195] at fold[6]
______________________________________________________
Start Training
Accuracy[0.7729] Recall[0.7284] F1[0.6898] at fold[7]

In [0]:
Run('data/SNOW/MHEALTH.npz')

Start Training
Epoch 00071: early stopping THR
Accuracy[0.8566] Recall[0.8323] F1[0.8400] at fold[0]
______________________________________________________
Start Training
Accuracy[0.8359] Recall[0.7883] F1[0.7508] at fold[1]
______________________________________________________
Start Training
Epoch 00063: early stopping THR
Accuracy[0.8633] Recall[0.8553] F1[0.8459] at fold[2]
______________________________________________________
Start Training
Epoch 00078: early stopping THR
Accuracy[0.8477] Recall[0.8239] F1[0.8235] at fold[3]
______________________________________________________
Start Training
Epoch 00062: early stopping THR
Accuracy[0.8196] Recall[0.8201] F1[0.8069] at fold[4]
______________________________________________________
Start Training
Epoch 00053: early stopping THR
Accuracy[0.8824] Recall[0.8557] F1[0.8593] at fold[5]
______________________________________________________
Start Training
Epoch 00068: early stopping THR
Accuracy[0.8039] Recall[0.7812] F1[0.7808] at fol

In [0]:
Run('data/FNOW/MHEALTH.npz')

Start Training
Epoch 00097: early stopping THR
Accuracy[0.7407] Recall[0.7386] F1[0.7297] at fold[0]
______________________________________________________
Start Training
Epoch 00095: early stopping THR
Accuracy[0.8741] Recall[0.8807] F1[0.8810] at fold[1]
______________________________________________________
Start Training
Accuracy[0.7463] Recall[0.7159] F1[0.6848] at fold[2]
______________________________________________________
Start Training
Accuracy[0.8134] Recall[0.7689] F1[0.7284] at fold[3]
______________________________________________________
Start Training
Accuracy[0.8060] Recall[0.7588] F1[0.7511] at fold[4]
______________________________________________________
Start Training
Accuracy[0.7970] Recall[0.7544] F1[0.7189] at fold[5]
______________________________________________________
Start Training
Accuracy[0.7970] Recall[0.7816] F1[0.7569] at fold[6]
______________________________________________________
Start Training
Accuracy[0.8421] Recall[0.7973] F1[0.7559] at fold[7]